In [435]:
import pandas as pd

In [436]:
csv_1 = pd.read_csv('causal(1).csv', index_col=0, sep='|', encoding='latin1')

In [437]:
csv_2 = pd.read_csv('causal(2).csv', index_col=0, sep='|', encoding='latin1')
csv_2

,premise,hypothesis,rationales
0,"The Parma trolleybus system (Italian: ""Rete fi...",The trolleybus system has over 2 urban routes,"1. The correct answer is ""contradiction"". The ..."
1,Alexandra Lendon Bastedo (9 March 1946 â 12 ...,Sharron Macready was a popular character throu...,"2. The correct answer is ""contradiction"". The ..."
2,Alexandra Lendon Bastedo (9 March 1946 â 12 ...,Bastedo didn't keep any pets because of her vi...,"3. The correct answer is ""neutral"". The premis..."
3,Alexandra Lendon Bastedo (9 March 1946 â 12 ...,Alexandra Bastedo was named by her mother.,"4. The correct answer is ""neutral"". The premis..."
4,Alexandra Lendon Bastedo (9 March 1946 â 12 ...,Bastedo cared for all the animals that inhabit...,"5. The correct answer is ""neutral"". The premis..."
...,...,...,...
9995,Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis..."
9996,The Three Musketeers is a 1993 Austrian-Americ...,Charlie Sheen was the highest paid actor in Th...,"7. The correct answer is ""neutral"". The premis..."
9997,San Andreas is a 2015 American disaster film d...,San Andreas was very expensive to produce.,"8. The correct answer is ""neutral"". The premis..."
9998,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika publishes her work in French.,"9. The correct answer is ""contradiction"". The ..."


In [438]:
csv_1 = pd.concat([csv_1, csv_2])
csv_1

,premise,hypothesis,rationales
10000,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika does not engage in the writing of ...,1. Contradiction. The premise clearly states t...
10001,"""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",2. Neutral. The premise provides information a...
10002,"Edward George ""Ed"" Skrein ( ; born 29 March 19...",The Transporter Refueled was a box-office succ...,3. Contradiction. The premise states that Ed S...
10003,"""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",4. Neutral. While the premise provides informa...
10004,Old Trafford is a tram stop on the Metrolink l...,Old Trafford Road road station typically serve...,5. Entailment. The premise states that the Old...
...,...,...,...
9995,Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis..."
9996,The Three Musketeers is a 1993 Austrian-Americ...,Charlie Sheen was the highest paid actor in Th...,"7. The correct answer is ""neutral"". The premis..."
9997,San Andreas is a 2015 American disaster film d...,San Andreas was very expensive to produce.,"8. The correct answer is ""neutral"". The premis..."
9998,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika publishes her work in French.,"9. The correct answer is ""contradiction"". The ..."


In [439]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [440]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
For the premise '%s' and hypothesis %s, what is the correct answer among the options "entailment", "neutral", "contradiction"? Explain the cause-and-effect relationship between the question and your chosen answer.\n
'''

In [441]:
error = csv_1[csv_1.rationales.isna()]

In [442]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [443]:
error.index

Index([], dtype='int64')

In [444]:
print(error.prompt[:10].values)

[]


In [445]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [446]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [447]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [450]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 
                   'contradictory', 'entail', 'contradict', 'supports', 'unrelated', 'discrepancy']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [451]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [452]:
csv_1[csv_1.LLM_answer.isna()].index

Index([], dtype='int64')

In [744]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [381]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [382]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [383]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [384]:
paper.shape

(5592, 6)

In [385]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 589},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradicts': 446},contradicts
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'contradicts': 484},contradicts
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entails': 519},entails
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'entail': 470},entail
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,{'neutral': 585},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 563},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 564},entailment


In [386]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'supports', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'unrelated', 'LLM_answer'] = 'neutral'
total_df.loc[total_df.LLM_answer == 'discrepancy', 'LLM_answer'] = 'contradiction'

In [387]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 589},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradicts': 446},contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'contradicts': 484},contradiction
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entails': 519},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'entail': 470},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,{'neutral': 585},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 563},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 564},entailment


In [388]:
paper['my_label'] = total_df['LLM_answer']

In [389]:
sum(paper.label == paper.my_label) / len(paper)

0.6183834048640916

In [390]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [935]:
total_df.to_csv('GPT_II - full.csv')